In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
def transformar_sexo(valor):
    if valor == 'female':
        return 1
    else:
        return 0
    
train['Sex_binario'] = train['Sex'].map(transformar_sexo)
test['Sex_binario'] = test['Sex'].map(transformar_sexo)

In [4]:
variaveis = ['Sex_binario', 'Age']

In [5]:
X = train[variaveis].fillna(-1)
Y = train['Survived']
X_prev = test[variaveis]
X_prev = X_prev.fillna(-1)

In [6]:
np.random.seed(0)
X_treino, X_valid, Y_treino, Y_valid = train_test_split(X, Y, test_size=0.5)

In [7]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
modelo = modelo.fit(X_treino,Y_treino)

In [ ]:
p = modelo.predict(X_valid)
np.mean(Y_valid == p)

In [ ]:
p = (X_valid['Sex_binario'] == 1).astype(np.int64)
np.mean(Y_valid == p)

## Validação Cruzada

In [10]:
from sklearn.model_selection import RepeatedKFold

In [ ]:
kf = RepeatedKFold(n_splits=2,n_repeats=10, random_state=10)
result = []
for linhas_treino, linhas_valid in kf.split(X):

    X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
    Y_treino, Y_valid = Y.iloc[linhas_treino], Y.iloc[linhas_valid]

    modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
    modelo = modelo.fit(X_treino,Y_treino)

    p = modelo.predict(X_valid)

    acc = np.mean(Y_valid == p)
    result.append(acc)
    print(acc)

In [ ]:
%pylab inline
pylab.hist(result)

In [ ]:
np.mean(result)

## Novas Variáveis Segundo Modelo

In [14]:
variaveis = ['Sex_binario', 'Age', 'Pclass','SibSp', 'Parch', 'Fare']

In [15]:
X = train[variaveis].fillna(-1)
Y = train['Survived']

In [ ]:
kf = RepeatedKFold(n_splits=2,n_repeats=10, random_state=10)
result = []
for linhas_treino, linhas_valid in kf.split(X):

    X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
    Y_treino, Y_valid = Y.iloc[linhas_treino], Y.iloc[linhas_valid]

    modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
    modelo = modelo.fit(X_treino,Y_treino)

    p = modelo.predict(X_valid)

    acc = np.mean(Y_valid == p)
    result.append(acc)
    print(acc)

In [ ]:
%pylab inline
pylab.hist(result)

## Retreinar modelo com todos os dados

In [18]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
modelo = modelo.fit(X,Y)

p = modelo.predict(test[variaveis].fillna(-1))

In [19]:
sub = pd.Series(p, index=test['PassengerId'], name='Survived')
sub.to_csv('segundo_modelo.csv',header=True)